In [6]:
import os

from typing_extensions import override

import secret_key
from judgeval.tracer import Tracer
from google import genai

gen_ai_client = genai.client.Client(api_key=secret_key.GEMINI_API_KEY)

In [7]:
judgment = Tracer(project_name="my_project", api_key=secret_key.JUDGMENT_API_KEY, organization_id=secret_key.JUDGMENT_ORG_ID)

@judgment.observe(span_type="tool")
def format_question(question: str) -> str:
    # dummy tool
    return f"Question : {question}"


@judgment.observe(span_type="function")
def run_agent(prompt: str) -> str:
    task = format_question(prompt)
    response = gen_ai_client.models.generate_content(model="gemini-2.5-pro", contents=task)
    return response.text


In [8]:
print(run_agent("What is the capital of the United States?"))

🔍 You can view your trace data here: ]8;id=326490;https://app.judgmentlabs.ai/app/project/1169b8d8-61ce-4e41-8378-7b9facf8cadc/monitor?trace_id=e1f4d80d-f6c2-419e-989d-88947140b3e2&show_trace=true\View Trace]8;;\

The capital of the United States is **Washington, D.C.**

"D.C." stands for the **District of Columbia**, a federal district created specifically to be the seat of the U.S. government.


In [9]:
from judgeval import JudgmentClient
from judgeval.data import Example
from judgeval.scorers import FaithfulnessScorer
client = JudgmentClient(api_key=secret_key.JUDGMENT_API_KEY, organization_id=secret_key.JUDGMENT_ORG_ID)
task = "What is the capital of the United States?"
example = Example(
    input=task,
    actual_output=run_agent(task),  # e.g. "The capital of the U.S. is Washington, D.C."
    retrieval_context=["Washington D.C. was founded in 1790 and became the capital of the U.S."],
)
scorer = FaithfulnessScorer(threshold=0.5)
client.assert_test(
    examples=[example],
    scorers=[scorer],
    model="gemini-2.5-pro",
)

2025-07-18 15:46:10 - judgeval - INFO - Successfully initialized JudgmentClient!


🔍 You can view your trace data here: ]8;id=418781;https://app.judgmentlabs.ai/app/project/1169b8d8-61ce-4e41-8378-7b9facf8cadc/monitor?trace_id=f407264b-0d3f-482a-8c7d-097aafb4eaa2&show_trace=true\View Trace]8;;\

🔍 You can view your evaluation results here: ]8;id=702764;https://app.judgmentlabs.ai/app/project/2969ae8a-574c-4955-b81b-e9e85899de93/experiment?eval_run_name=68becfad-7e03-4488-b482-3bc6905a5371\View Results]8;;\

================================================================================

⚠️  TEST RESULTS: 0/1 passed (1 failed)

...

  Reason: 

  Error: 'exc_info' is an invalid keyword argument for print()

----------------------------------------

================================================================================

AssertionError: [{'failed_scorers': [ScorerDataJudgmentType(name='Faithfulness', threshold=0.5, success=False, score=None, reason='', strict_mode=False, evaluation_model='gemini-2.5-pro', error="'exc_info' is an invalid keyword argument for print()", additional_metadata=None)]}]

In [13]:
from judgeval.scorers import AnswerCorrectnessScorer

client = JudgmentClient(api_key=secret_key.JUDGMENT_API_KEY, organization_id=secret_key.JUDGMENT_ORG_ID)
task = "What is the capital of the United States?"
example = Example(
    input=task,
    actual_output=run_agent(task),  # e.g. "The capital of the U.S. is Washington, D.C."
    retrieval_context=["Washington D.C. was founded in 1790 and became the capital of the U.S."],
)
scorer = AnswerCorrectnessScorer(threshold=0.5)
client.assert_test(
    examples=[example],
    scorers=[scorer],
    model="gemini-2.5-pro",
    override=True
)

🔍 You can view your trace data here: ]8;id=785252;https://app.judgmentlabs.ai/app/project/1169b8d8-61ce-4e41-8378-7b9facf8cadc/monitor?trace_id=744d5aae-4878-45c3-bdf8-3f81f4f780bc&show_trace=true\View Trace]8;;\

⚠️  WARNING: Example is missing required parameters for scorer Answer Correctness

Missing parameters: expected_output

Example: {
  "input": "What is the capital of the United States?",
  "actual_output": "The capital of the United States is **Washington, D.C.**\n\n\"D.C.\" stands for **District of 
Columbia**.",
  "expected_output": null,
  "context": null,
  "retrieval_context": [
    "Washington D.C. was founded in 1790 and became the capital of the U.S."
  ],
  "additional_metadata": null,
  "tools_called": [],
  "expected_tools": [],
  "name": null,
  "example_id": null,
  "example_index": 0,
  "created_at": "2025-07-18T15:59:59.445811",
  "trace_id": null,
  "trace_span_id": null,
  "dataset_id": null
}

----------------------------------------

SystemExit: 0

/opt/anaconda3/envs/genai/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
